In [20]:
from konlpy.tag import Okt
import pickle
import numpy as np

In [21]:
from konlpy.tag import Okt
okt = Okt()

def tokenize(text) :
    result = []
    word_s = okt.pos(text, norm=True, stem=True)
    for n, h in word_s :
        if not (h in ['Noun', 'Verb', 'Adjective']) : continue
        result.append(n)
    return result

In [22]:
#'_id' : 전체 id의 갯수
word_dic = {'_id' : 0} 

def words_to_ids(words) :
    result = []
    for w in words :
        if w in word_dic :
            result.append(word_dic[w])
            continue
        else :
            word_dic[w] = word_dic['_id']
            #전체 id의 갯수 기록(몇 번째 id인지 알아야하니까!)
            word_dic['_id'] += 1 
            result.append(word_dic[w])
    return result         

In [23]:
files = []

def add_text(text) :
    ids = words_to_ids(tokenize(text))
    files.append(ids)

In [24]:
def calc_files() :
    global dt_dic
    result = []
    doc_count = len(files)
    dt_dic = {} #문장 전체에서의 단어 출현 횟수
    
    
    #단어 출현 횟수 세기
    for words in files :
        used_word = {}
        data = np.zeros(word_dic['_id'])
        for id in words :
            data[id] += 1
            used_word[id] = 1
        #단어 t가 사용되고 있을 경우 dt_dic의 수를 1 더하기
        for id in used_word :
            if not(id in dt_dic) : dt_dic[id] = 0
            dt_dic[id] += 1
        #정규화하기
        data = data/len(words)
        result.append(data)
    
    #TF-IDF 계산하기
    for i, doc in enumerate(result) :
        for id, v in enumerate(doc) :
            idf = np.log(doc_count/dt_dic[id]) + 1
            doc[id] = min([doc[id]*idf, 1.0])
        result[i] = doc
    return result

In [16]:
if __name__ == '__main__' :
    add_text('비')
    add_text('오늘은 비가 내렸어요.')
    add_text('오늘은 더웠지만 오후부터 비가 내렸다.')
    add_text('비가 내리는 일요일이다.')
    print("---------------------------------")
    print("word_dic : " , word_dic)
    print("---------------------------------")
    print("files : " , files)

---------------------------------
word_dic :  {'오늘': 1, '비': 0, '일요일': 5, '_id': 6, '덥다': 3, '오후': 4, '내리다': 2}
---------------------------------
files :  [[0], [1, 0, 2], [1, 3, 4, 0, 2], [0, 2, 5]]


In [17]:
print("word_dic : " , word_dic)
print("---------------------------------")
print(calc_files())

word_dic :  {'오늘': 1, '비': 0, '일요일': 5, '_id': 6, '덥다': 3, '오후': 4, '내리다': 2}
---------------------------------
[array([1., 0., 0., 0., 0., 0.]), array([0.33333333, 0.56438239, 0.42922736, 0.        , 0.        ,
       0.        ]), array([0.2       , 0.33862944, 0.25753641, 0.47725887, 0.47725887,
       0.        ]), array([0.33333333, 0.        , 0.42922736, 0.        , 0.        ,
       0.79543145])]


In [25]:
def add_file(path) :
    with open(path, "r", encoding="utf-8") as f :
        s = f.read()
        add_text(s)

In [26]:
import glob

x= []
y= []

def read_files(path, label) :
    print("read_files : ", path)
    files = glob.glob(path + "/*.txt")
    for f in files :
        add_file(f)
        y.append(label)
        
read_files('text/100',0)
read_files('text/101',1)
read_files('text/103',2)
read_files('text/105',3)

x = calc_files()

read_files :  text/100
read_files :  text/101
read_files :  text/103
read_files :  text/105


In [27]:
import pickle

def save_dic(fname) :
    pickle.dump(
    [word_dic, dt_dic, files],
    open(fname, "wb"))

pickle.dump([y,x], open('genre.pickle', 'wb'))
save_dic('genre-tfidf.dic')
print('ok')

ok


In [31]:
print(x)

[array([0.05599533, 0.03647884, 0.02345133, ..., 0.        , 0.        ,
       0.        ]), array([0.05330424, 0.02083542, 0.01339457, ..., 0.        , 0.        ,
       0.        ]), array([0., 0., 0., ..., 0., 0., 0.]), array([0.01848564, 0.00903202, 0.        , ..., 0.        , 0.        ,
       0.        ]), array([0., 0., 0., ..., 0., 0., 0.]), array([0.00843205, 0.        , 0.        , ..., 0.        , 0.        ,
       0.        ]), array([0.04871215, 0.        , 0.        , ..., 0.        , 0.        ,
       0.        ]), array([0.08512943, 0.        , 0.        , ..., 0.        , 0.        ,
       0.        ]), array([0.01135338, 0.        , 0.        , ..., 0.        , 0.        ,
       0.        ]), array([0.01096487, 0.        , 0.        , ..., 0.        , 0.        ,
       0.        ]), array([0.04179362, 0.        , 0.        , ..., 0.        , 0.        ,
       0.        ]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0.0

In [30]:
print(y)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [33]:
import keras
y = keras.utils.np_utils.to_categorical(y,4)
print(y)

[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]]
